In [1]:
from sympy import *
from long_functions import *
import math
init_printing(use_unicode=True)

In [2]:
# STATE
x = Symbol('x')
y = Symbol('y')
theta = Symbol('theta')
l_ratio = Symbol('l_ratio')
d_ratio = Symbol('d_ratio')
h = Symbol('h')
v = Symbol('v')
w = Symbol('w')
state = Matrix([x,y,theta,l_ratio,d_ratio,h,v,w])

l = l_ratio*h
d = d_ratio*h

# VIEW MATRIX
# vr00 = Symbol('Vr_00')
# vr01 = Symbol('Vr_01')
# vr02 = Symbol('Vr_02')
# vr10 = Symbol('Vr_10')
# vr11 = Symbol('Vr_11')
# vr12 = Symbol('Vr_12')
# vr20 = Symbol('Vr_20')
# vr21 = Symbol('Vr_21')
# vr22 = Symbol('Vr_22')
# VR = Matrix([[vr00, vr01, vr02],[vr10, vr11, vr12],[vr20, vr21, vr22]])
# vtx = Symbol('Vtx')
# vty = Symbol('Vty')
# vtz = Symbol('Vtz')
vr00 = Symbol('vr00')
vr01 = Symbol('vr01')
vr02 = Symbol('vr02')
vr10 = Symbol('vr10')
vr11 = Symbol('vr11')
vr12 = Symbol('vr12')
vr20 = Symbol('vr20')
vr21 = Symbol('vr21')
vr22 = Symbol('vr22')
VR = Matrix([[vr00, vr01, vr02],[vr10, vr11, vr12],[vr20, vr21, vr22]])
vtx = Symbol('vtx')
vty = Symbol('vty')
vtz = Symbol('vtz')
VT = Matrix([vtx,vty,vtz])
viewMat = eye(4)
viewMat[0:3,0:3] = VR
viewMat[0:3,3] = VT

# PROJECTION MATRIX
fx = Symbol('fx')
fy = Symbol('fy')
cx = Symbol('cx')
cy = Symbol('cy')
P = Matrix([[fx,0,cx,0],[0,fy,cy,0],[0,0,1,0]])

# WORLD VIEW PROJECTION MATRIX
m00 = Symbol('m_00')
m01 = Symbol('m_01')
m02 = Symbol('m_02')
m03 = Symbol('m_03')
m10 = Symbol('m_10')
m11 = Symbol('m_11')
m12 = Symbol('m_12')
m13 = Symbol('m_13')
m20 = Symbol('m_20')
m21 = Symbol('m_21')
m22 = Symbol('m_22')
m23 = Symbol('m_23')
M_wvp = Matrix([[m00, m01, m02, m03],[m10, m11, m12, m13],[m20, m21, m22, m23]])

In [3]:
# ELLIPSOID
z = Symbol('z')

a = Symbol('a_3D')
b = Symbol('b_3D')
c = Symbol('c_3D')

subs_ellipsoid = {
    z : h/2,
    a : l/2,
    b : d/2,
    c : h/2
}

Q = Matrix([[a**2, 0, 0, 0], [0, b**2, 0, 0],[0, 0, c**2, 0],[0, 0, 0, -1]])

# WORLD Matrix
WR = Matrix([[cos(theta), -sin(theta), 0],[sin(theta), cos(theta), 0],[0,0,1]])
Re_w = eye(4)
Re_w[0:3,0:3] = WR.T

center_ellipsoid = Matrix([x,y,z])
T_c_inv = eye(4)
T_c_inv[0:3,3] = center_ellipsoid

W = T_c_inv*Re_w.T
# world view projection matrix
M_wvp_real = P*viewMat*W

subs_wvp = {
    m00 : M_wvp_real[0,0],
    m01 : M_wvp_real[0,1],
    m02 : M_wvp_real[0,2],
    m03 : M_wvp_real[0,3],
    m10 : M_wvp_real[1,0],
    m11 : M_wvp_real[1,1],
    m12 : M_wvp_real[1,2],
    m13 : M_wvp_real[1,3],
    m20 : M_wvp_real[2,0],
    m21 : M_wvp_real[2,1],
    m22 : M_wvp_real[2,2],
    m23 : M_wvp_real[2,3]
}

# Compute the check to perform to see if an object is behind the camera
M_wv = viewMat*W
Q_star_camera = M_wv * Q * M_wv.T
check = Q_star_camera[2,3]/Q_star_camera[3,3]
# if check < 0 the object is behind the camera 

Matrix([[cos(theta), sin(theta), 0, 0], [-sin(theta), cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
Matrix([[cos(theta), -sin(theta), 0, x], [sin(theta), cos(theta), 0, y], [0, 0, 1, z], [0, 0, 0, 1]])


In [4]:
# ELLIPSE
C_star = M_wvp * Q * M_wvp.T

C_star = (-1/C_star[2,2])*C_star
ellipse_centre = Matrix([-C_star[0,2],-C_star[1,2]])

T_e_origin = eye(3)
T_e_origin[0:2, 2] = -ellipse_centre
C_centre = T_e_origin*C_star*T_e_origin.T
C_centre = 0.5*(C_centre + C_centre.T)

C_submat = C_centre[0:2, 0:2]

c00 = Symbol('c_00')
c01 = Symbol('c_01')
c10 = Symbol('c_10')
c11 = Symbol('c_11')
# Substitutions to reduce complexity
subs_c = {
    c00: C_submat[0,0],
    c01: C_submat[0,1],
    c10: C_submat[1,0],
    c11: C_submat[1,1],
}

temp = Matrix([[c00,c01], [c10,c11]])

lambdas = list(temp.eigenvals().keys())
lambdas_list = [sqrt(l) for l in lambdas]
semi_axes = Matrix(lambdas_list)

semi_axes = semi_axes.subs(subs_c)
# Simplify the formula for the axis, this is the maximum depth for semplification, 
# do that on other terms require too much time or do not have effect
semi_axes.simplify()
# [X_center, Y_center, semi_axis_A, semi_axis_B]
ellipse_state = Matrix([ellipse_centre[0], ellipse_centre[1], semi_axes[0], semi_axes[1]])


# Here we can obtain the final equations
ellipse_state = ellipse_state.subs(subs_wvp).subs(subs_ellipsoid)

In [ ]:
print(pycode(C_submat.subs(subs_wvp).subs(subs_ellipsoid)))

ImmutableDenseMatrix([[vr00*(-1/2*h*vr02*x + vr00*((1/4)*d_ratio**2*h**2*math.sin(theta)**2 + (1/4)*h**2*l_ratio**2*math.cos(theta)**2 - x**2) + vr01*(-1/4*d_ratio**2*h**2*math.sin(theta)*math.cos(theta) + (1/4)*h**2*l_ratio**2*math.sin(theta)*math.cos(theta) - x*y) - vtx*x) + vr01*(-1/2*h*vr02*y + vr00*(-1/4*d_ratio**2*h**2*math.sin(theta)*math.cos(theta) + (1/4)*h**2*l_ratio**2*math.sin(theta)*math.cos(theta) - x*y) + vr01*((1/4)*d_ratio**2*h**2*math.cos(theta)**2 + (1/4)*h**2*l_ratio**2*math.sin(theta)**2 - y**2) - vtx*y) + vr02*(-1/2*h*vr00*x - 1/2*h*vr01*y - 1/2*h*vtx) + vtx*(-1/2*h*vr02 - vr00*x - vr01*y - vtx), vr10*(-1/2*h*vr02*x + vr00*((1/4)*d_ratio**2*h**2*math.sin(theta)**2 + (1/4)*h**2*l_ratio**2*math.cos(theta)**2 - x**2) + vr01*(-1/4*d_ratio**2*h**2*math.sin(theta)*math.cos(theta) + (1/4)*h**2*l_ratio**2*math.sin(theta)*math.cos(theta) - x*y) - vtx*x) + vr11*(-1/2*h*vr02*y + vr00*(-1/4*d_ratio**2*h**2*math.sin(theta)*math.cos(theta) + (1/4)*h**2*l_ratio**2*math.sin(theta

In [ ]:
print(pycode(ellipse_state))

In [ ]:
# H MATRIX
jacobian = ellipse_state.jacobian(state)
print(pycode(jacobian))
# impossible to simplify
# H_matrix = jacobian.simplify()

In [ ]:
# With print is possible to print the code
# print(pycode(ax_a))
print("\\\\result is a eigen::Matrix<4,8,RowMajor>, delete last ,\nresult <<")
for i in ellipse_state:
    print("\t",ccode(i), ",")
print(";")

\\result is a eigen::Matrix<4,8,RowMajor>, delete last ,
result <<
	 ((1.0/4.0)*pow(d_ratio, 2)*pow(h, 2)*(-vr20*sin(theta) + vr21*cos(theta))*(-(cx*vr20 + fx*vr00)*sin(theta) + (cx*vr21 + fx*vr01)*cos(theta)) + (1.0/4.0)*pow(h, 2)*pow(l_ratio, 2)*(vr20*cos(theta) + vr21*sin(theta))*((cx*vr20 + fx*vr00)*cos(theta) + (cx*vr21 + fx*vr01)*sin(theta)) + (1.0/4.0)*pow(h, 2)*vr22*(cx*vr22 + fx*vr02) - ((1.0/2.0)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1.0/2.0)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1.0/4.0)*pow(d_ratio, 2)*pow(h, 2)*pow(-vr20*sin(theta) + vr21*cos(theta), 2) + (1.0/4.0)*pow(h, 2)*pow(l_ratio, 2)*pow(vr20*cos(theta) + vr21*sin(theta), 2) + (1.0/4.0)*pow(h, 2)*pow(vr22, 2) - pow((1.0/2.0)*h*vr22 + vr20*x + vr21*y + vtz, 2)) ,
	 ((1.0/4.0)*pow(d_ratio, 2)*pow(h, 2)*(-vr20*sin(theta) + vr21*cos(theta))*(-(cy*vr20 + fy*vr10)*sin(theta) + (cy*vr21 + fy*vr11)*cos(theta)) + (1.0/4.0)*pow(h, 2)*pow(l_ratio, 2)*(vr20*cos(theta) + vr21*sin

In [ ]:
# print in latex format the formulas for the world view projection matrix
for i in range(3):
    for j in range(4):
        pass
        #print("m_{{{}{}}} &= ".format(i,j), latex(M_wvp_real[i,j]), "\\\\")

In [ ]:
# test values taken from simulation of an object inside the view image
subs_values = {
    x :         -23.2729,
    y :         19.1563,
    theta :     1.90237,
    l_ratio :   0.167577,
    d_ratio :   0.231017,
    h :         1.86,
    vr00 :      0.163513,
    vr01 :      0.986458,
    vr02 :      0.0127917,
    vr10 :      0.32056,
    vr11 :      -0.0408638,
    vr12 :      -0.946346,
    vr20 :      -0.933008,
    vr21 :      0.15884,
    vr22 :      -0.322901,
    vtx :       -1.06324,
    vty :       6.26204,
    vtz :       1.73534,
    fx :        295.6,
    fy :        295.6,
    cx :        512,
    cy :        256
}

# dictionary to use for testing, decompose it to pass it to the functions
function_values = {
    'x' :         -29.0,
    'y' :         16.0,
    'theta' :     0.0,
    'l_ratio' :   1.0,
    'd_ratio' :   1.0,
    'h' :         2.0,
    'vr00' :      0.163513,
    'vr01' :      0.986458,
    'vr02' :      0.0127917,
    'vr10' :      0.32056,
    'vr11' :      -0.0408638,
    'vr12' :      -0.946346,
    'vr20' :      -0.933008,
    'vr21' :      0.15884,
    'vr22' :      -0.322901,
    'vtx' :       -1.06324,
    'vty' :       6.26204,
    'vtz' :       1.73534,
    'fx' :        295.6,
    'fy' :        295.6,
    'cx' :        512.0,
    'cy' :        256.0
}

In [ ]:
# Right
print(compute_Q_star_fix(**function_values))
print(compute_Q_star(**function_values))
print(compute_C_star(**function_values))
# To check
print(compute_C_sub(**function_values))
print(compute_ellipse_state(**function_values))

Matrix([[-840.000000000000, 464.000000000000, 29.0000000000000, 29.0000000000000], [464.000000000000, -255.000000000000, -16.0000000000000, -16.0000000000000], [29.0000000000000, -16.0000000000000, 0, -1.00000000000000], [29.0000000000000, -16.0000000000000, -1.00000000000000, -1]])
Matrix([[-98.8201354368854, 46.3019711423218, -309.832094132105, -9.99100270000000], [46.3019711423218, -20.4773563214758, 143.716863468789, 4.63436680000000], [-309.832094132105, 143.716863468789, -960.689006324856, -31.0111110000000], [-9.99100270000000, 4.63436680000000, -31.0111110000000, -1]])
Matrix([[-354258138.897673, -123568557.911589, -583459.138263777], [-123568557.911589, -42997868.1295848, -203453.680777789], [-583459.138263777, -203453.680777789, -960.689006324856]])
Matrix([[-368754.234268692, -128624.931791719], [-128624.931791719, -44757.3229697656]])


In [ ]:
# Test function with values
for i in range(10):
    compute_jacobian(
        x =         i,
        y =         19.1563,
        theta =     1.90237,
        l_ratio =   0.167577,
        d_ratio =   0.231017,
        h =         1.86,
        vr00 =      0.163513,
        vr01 =      0.986458,
        vr02 =      0.0127917,
        vr10 =      0.32056,
        vr11 =      -0.0408638,
        vr12 =      -0.946346,
        vr20 =      -0.933008,
        vr21 =      0.15884,
        vr22 =      -0.322901,
        vtx =       -1.06324,
        vty =       6.26204,
        vtz =       1.73534,
        fx =        295.6,
        fy =        295.6,
        cx =        512,
        cy =        256
        )
